### Generar tonos simples con numpy y reproducir con sounddevice
Esta opción es útil si quieres generar ondas sonoras básicas como senoidales, cuadradas, etc.

In [1]:
import numpy as np
import sounddevice as sd


In [2]:
def generate_tone(frequency, duration, sample_rate=44100):
    """
    Genera un tono senoidal.
    frequency: Frecuencia del tono en Hz.
    duration: Duración en segundos.
    sample_rate: Frecuencia de muestreo.
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    tone = 0.5 * np.sin(2 * np.pi * frequency * t)  # Amplitud de 0.5 para evitar distorsión
    return tone

In [3]:
# Genera un tono de 440 Hz (nota La) durante 2 segundos
tone = generate_tone(frequency=440, duration=2)
sd.play(tone, samplerate=44100)  # Reproduce el tono
sd.wait()  # Espera hasta que termine

### Crear un archivo WAV con wave

In [4]:
import wave

In [5]:
def generate_tone_wav(frequency, duration, sample_rate=44100, file_name="tone.wav"):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    tone = (0.5 * np.sin(2 * np.pi * frequency * t) * 32767).astype(np.int16)  # Escalado a 16 bits
    with wave.open(file_name, 'w') as wav_file:
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 16 bits por muestra
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(tone.tobytes())

In [13]:
generate_tone_wav(frequency=700, duration=2, file_name="data/audios_generados/output.wav")


In [14]:
import IPython.display as ipd

audio_file = 'data/audios_generados/output.wav'

ipd.Audio(audio_file) 

### Crear audios con soundfile

In [16]:
import soundfile as sf

def save_tone_with_soundfile(frequency, duration, file_name="tone.wav", sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    tone = 0.5 * np.sin(2 * np.pi * frequency * t)  # Amplitud de 0.5
    sf.write(file_name, tone, samplerate=sample_rate)

# Guarda un tono de 440 Hz
save_tone_with_soundfile(frequency=440, duration=2, file_name="data/audios_generados/output2.wav")


In [17]:
audio_file = 'data/audios_generados/output2.wav'
ipd.Audio(audio_file)

### GENERAR un acorde

In [18]:
def generate_chord(frequencies, duration, file_name="chord.wav", sample_rate=44100):
    '''
    genera un acorde a partir de varias frecuencias.
    frequencies: lista de frecuencias en hz.
    sample_rate: ffrecuencia de muestreo.
    '''
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    chord = sum(0.5 * np.sin(2 * np.pi * freq * t) for freq in frequencies)
    chord /= len(frequencies)  
    sf.write(file_name, chord, samplerate=sample_rate)

In [20]:
# Genera un acorde mayor (Do, Mi, Sol)
generate_chord(frequencies=[261.63, 329.63, 392.00], duration=2, file_name="data/audios_generados/chord.wav")

In [21]:
file_name="data/audios_generados/chord.wav"
ipd.Audio(file_name)

In [22]:
generate_chord(frequencies=[261.63, 311.13, 392.00], duration=2, file_name="data/audios_generados/do_menor.wav")


In [23]:
do_menor="data/audios_generados/do_menor.wav"
ipd.Audio(do_menor)

In [26]:
def generate_fm_tone(carrier_freq, mod_freq, mod_index, duration, file_name="fm_tone.wav", sample_rate=44100):
    """
    Genera un tono con modulación de frecuencia (FM Synthesis).
    carrier_freq: Frecuencia portadora.
    mod_freq: Frecuencia moduladora.
    mod_index: Índice de modulación.
    duration: Duración en segundos.
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    modulator = mod_index * np.sin(2 * np.pi * mod_freq * t)  # Onda moduladora
    fm_tone = 0.5 * np.sin(2 * np.pi * carrier_freq * t + modulator)  # FM Synthesis
    sf.write(file_name, fm_tone, samplerate=sample_rate)


In [ ]:
# Genera un tono FM
generate_fm_tone(carrier_freq=440, mod_freq=50, mod_index=75, duration=2, file_name="data/audios_generados/fm_tone.wav")


In [30]:
file_name="data/audios_generados/fm_tone.wav"
ipd.Audio(file_name)

In [28]:
def generate_tone_with_adsr(frequency, duration, adsr, file_name="adsr_tone.wav", sample_rate=44100):
    """
    Genera un tono con envolvente ADSR.
    frequency: Frecuencia del tono.
    duration: Duración en segundos.
    adsr: Diccionario con las fases de la envolvente (Attack, Decay, Sustain, Release).
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    tone = 0.5 * np.sin(2 * np.pi * frequency * t)
    
    # Genera la envolvente ADSR
    attack_time, decay_time, sustain_level, release_time = adsr.values()
    attack_samples = int(attack_time * sample_rate)
    decay_samples = int(decay_time * sample_rate)
    release_samples = int(release_time * sample_rate)
    sustain_samples = len(t) - (attack_samples + decay_samples + release_samples)
    
    envelope = np.concatenate([
        np.linspace(0, 1, attack_samples),  # Attack
        np.linspace(1, sustain_level, decay_samples),  # Decay
        np.full(sustain_samples, sustain_level),  # Sustain
        np.linspace(sustain_level, 0, release_samples)  # Release
    ])
    
    # Aplica la envolvente al tono
    adsr_tone = tone[:len(envelope)] * envelope
    sf.write(file_name, adsr_tone, samplerate=sample_rate)


In [29]:

# Genera un tono con envolvente ADSR
generate_tone_with_adsr(
    frequency=440,
    duration=2,
    adsr={"Attack": 0.1, "Decay": 0.1, "Sustain": 0.7, "Release": 0.2},
    file_name="data/audios_generados/adsr_tone.wav"
)


In [31]:
file_name="data/audios_generados/adsr_tone.wav"
ipd.Audio(file_name)

### pianito

In [37]:
def generate_note(frequency, duration, sample_rate=44100, amplitude=0.5):
    """
    Genera una onda senoidal para una nota específica.
    frequency: Frecuencia de la nota (Hz).
    duration: Duración de la nota (segundos).
    sample_rate: Frecuencia de muestreo.
    amplitude: Amplitud del sonido.
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    return amplitude * np.sin(2 * np.pi * frequency * t)

def generate_melody(notes, durations, file_name="melody.wav", sample_rate=44100):
    """
    Genera una sucesión de notas (melodía).
    notes: Lista de frecuencias de las notas (Hz).
    durations: Lista de duraciones de las notas (segundos).
    file_name: Nombre del archivo de salida.
    sample_rate: Frecuencia de muestreo.
    """
    melody = np.array([])

    for freq, dur in zip(notes, durations):
        note = generate_note(freq, dur, sample_rate)
        melody = np.concatenate((melody, note))  # Concatenar notas

    # Normaliza la amplitud final
    melody /= np.max(np.abs(melody))

    # Guarda la melodía como archivo WAV
    sf.write(file_name, melody, samplerate=sample_rate)



In [38]:
# Frecuencias en Hz de las notas: Do, Re, Mi, Fa, Sol, La, Si, Do (C4-B4)
notes = [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25]
durations = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0]  # Duración de cada nota en segundos

# Genera la melodía
generate_melody(notes, durations, file_name="data/audios_generados/melody.wav")


In [39]:
melodia ="data/audios_generados/melody.wav"
ipd.Audio(melodia)